In [1]:
import os
import glob
import cv2
from tqdm import tqdm
import numpy as np
import random
import shutil
from PIL import Image
from multiprocessing import Pool

In [2]:
#source_path = './UCF101/UCF-101'
#output_path = "UCF101"
source_path = './dataset/RealLifeViolence/dataset'
output_path = "dataset/RealLifeViolence"
dim=(200,200)

In [3]:
def arrange_picture(picture: list):

    h1= np.hstack(((cv2.resize(picture[0], dim)), (cv2.resize(picture[1], dim)), (cv2.resize(picture[4], dim))))
    h2= np.hstack(((cv2.resize(picture[2], dim)), (cv2.resize(picture[3], dim)), (cv2.resize(picture[6], dim))))
    h3= np.hstack(((cv2.resize(picture[5], dim)), (cv2.resize(picture[7], dim)), (cv2.resize(picture[8], dim))))
    
    
    
    

    return np.vstack((h1, h2, h3))

def read_video_from_path(path):
    vidcap = cv2.VideoCapture(path)
    success, image = vidcap.read()
    count = 0
    picture_list=[]
    while success:
        if count % 1 == 0:
            picture_list.append(image)
        success, image = vidcap.read()
        count += 1
    return picture_list, count 

In [4]:
def save_to_frame(input_dir,  label,n_superpictures=6 ,n_picture_for_frame=9, name_dataset=""):
    id_training=0
    id_evaluation=0
    paths=glob.glob(input_dir + '/*')
    paths.sort()
    pbar= tqdm(paths)
    for path in pbar:
        pbar.set_postfix_str(path)
       # print(path)
        fname = os.path.basename(path).split('.')[0]
        n_total_frames =n_superpictures*n_picture_for_frame
      
        picture_list, count = read_video_from_path(path)
        
        pace = int(count/n_total_frames)
        index= range(n_total_frames)
        index= [value *pace for value in index]
        super_images_list=[]
        count =0
        

        if(np.random.uniform(0,1)< 0.8):
            split= "training" 
        else:
            split= "evaluation" 
        
        save_path=f"{name_dataset}/{split}/{label}"
        os.makedirs(save_path, exist_ok=True)
        
      #  tqdm.write(path)
        #print(path)
        for i in index:
            super_images_list.append(picture_list[i])
            count+=1
            if(count== n_picture_for_frame):
                results= arrange_picture(super_images_list)
                
                if(split=='training'):
                    final_path= f"{save_path}/{id_training}.jpg"
                    id_training+=1   
                else:
                    final_path= f"{save_path}/{id_evaluation}.jpg"
                    id_evaluation+=1
              
                cv2.imwrite(final_path, results)
                super_images_list=[]
                count=0


In [5]:

labels= os.listdir(source_path)
def proced_dataset(label):
    input_dir = source_path + "/" + label
    save_to_frame(input_dir, label ,name_dataset=output_path)
    return label

with Pool() as pool:
    for result in tqdm(pool.imap(proced_dataset, labels)):
        print(f"processed {result}")

100%|██████████| 994/994 [02:46<00:00,  5.97it/s, ./RealLifeViolence/dataset/Violence/V_999.mp4]
2it [02:46, 83.26s/it] 

processed Violence
processed NonViolence


In [6]:

# labels= os.listdir(source_path)
# def proced_dataset(label):
#     input_dir = source_path + "/" + label
#     save_to_frame(input_dir, label ,name_dataset=output_path)
#     return label

# for label in labels :
#     proced_dataset(label)
    